In [7]:
import os, json
import openai
import wandb

keys = json.load(".KEYS.json")
openai.api_key = keys['OPENAI']

In [343]:
names = ['Corgichu']

In [ ]:
gpt_prompt = "One nonsense word as a name of a creature"

while len(names) < 106:
    response = openai.Completion.create(
      engine="text-davinci-002",
      prompt=gpt_prompt,
      temperature=1,
      max_tokens=256,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )['choices'][0]['text'].strip()
    if response in names:
        print("duplicate", response)
        i -= 1
    else:
        good_response = input(response)
        if good_response != 'd':
            if len(good_response) == 0:
                names.append(response)
            else:
                names.append(good_response)

In [ ]:
removes = []
for idx, name in enumerate(names):
    new_name = input(name)
    if len(new_name) > 0:
        if new_name == 'd':
            removes.append(idx)
        else:
            names[idx] = new_name

In [381]:
names = [n.replace('\n', '').replace(':', '').replace('.', '') for i, n in enumerate(names)]

In [388]:
names = list(set(names))

In [389]:
len(names)

106

In [ ]:
personalities = []

In [ ]:
gpt_prompt = "Describe the personality of a creature as if it was an rpg character"

while len(personalities) < 106:
    response = openai.Completion.create(
      engine="text-davinci-002",
      prompt=gpt_prompt,
      temperature=1,
      max_tokens=256,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )['choices'][0]['text'].strip()
    
    if response in personalities:
        print("duplicate", response)
        i -= 1
    else:
        good_response = input(response)
        if good_response != 'd':
            if len(good_response) == 0:
                personalities.append(response)
            else:
                personalities.append(good_response)

In [392]:
personalities = list(set(personalities))

In [ ]:
removes = []
for idx, name in enumerate(personalities):
    new_name = input(name)
    if len(new_name) > 0:
        if new_name == 'd':
            removes.append(idx)
        else:
            personalities[idx] = new_name
personalities = [n for i, n in enumerate(personalities) if i not in removes]

In [455]:
for i, p in enumerate(personalities):
    if len(p) > 380:
        personalities[i] = '.'.join(p.split('.')[:-1]).strip()
        print(len(p), len(personalities[i]))

In [605]:
traits = {name: personality for name, personality in zip(names, personalities)}

In [606]:
json.dump(traits, open("traits.json", "w"))

In [427]:
from wand.color import Color
from wand.drawing import Drawing
from wand.image import Image
from textwrap import wrap

def word_wrap(image, ctx, text, roi_width, roi_height):
    """Break long text to multiple lines, and reduce point size
    until all text fits within a bounding box."""
    mutable_message = text
    iteration_attempts = 100

    def eval_metrics(txt):
        """Quick helper function to calculate width/height of text."""
        metrics = ctx.get_font_metrics(image, txt, True)
        return (metrics.text_width, metrics.text_height)

    def shrink_text():
        """Reduce point-size & restore original text"""
        ctx.font_size = ctx.font_size - 0.75
        mutable_message = text

    while ctx.font_size > 0 and iteration_attempts:
        iteration_attempts -= 1
        width, height = eval_metrics(mutable_message)
        if height > roi_height:
            shrink_text()
        elif width > roi_width:
            columns = len(mutable_message)
            while columns > 0:
                columns -= 1
                mutable_message = '\n'.join(wrap(mutable_message, columns))
                wrapped_width, _ = eval_metrics(mutable_message)
                if wrapped_width <= roi_width:
                    break
            if columns < 1:
                shrink_text()
        else:
            break
    if iteration_attempts < 1:
        raise RuntimeError("Unable to calculate word_wrap for " + text)
    return mutable_message

In [609]:
def get_card(img_path, save_path, i, name, personality, background = '#20b2aa', W=630, H=880):
#     name = name.upper()
    text = personality

    with Image(width=W, height=H, background=Color(background)) as image:    
        with Drawing() as draw:
            draw.text_alignment = 'center'
            # Name
#             draw.font = './Changa/Changa-VariableFont_wght.ttf'
            draw.font = './DayPosterBlack/DAYPBL__.ttf'
            draw.font_size = 50
            draw.push()
            draw.fill_color = Color('black')
            draw.text(int(W/2), 48, name)
            draw.pop()
            draw.push()
            
#             draw.font_size = 55
#             draw.fill_color = Color('black')
#             draw.text(int(W/2), 48, name)
#             draw.pop()
#             draw.push()
            
            # -fill white
            draw.font_size = 50
            draw.fill_color = Color('#FF7F50')
            draw.text(int(W/2) - 4, 48 -2, name)
            draw.pop()
            draw(image)

        BOX_X_PADDING = 6
        fg_img = Image(filename=img_path)
        with Image(width=fg_img.width, height=int(fg_img.height/2 - 20), background=Color(background)) as description:   
            with Drawing() as draw:
                # Box
                draw.fill_color = Color('#c9ffe5')
                draw.rectangle(left=0, top=0, right=fg_img.width, bottom=fg_img.height/2)

                with Drawing() as draw2:
                    # Box
                    draw2.fill_color = Color('white')
                    draw2.rectangle(left=0, top=0, right=fg_img.width, bottom=fg_img.height/2)
                    draw2.composite(operator='color_burn', left=0, top=0, width=description.width, height=description.height, image=description)

                # Text
                draw.font_size = 26
#                 draw.text_alignment = 'left'
                
#                 draw.text_interline_spacing = 0
                draw.font = './SF-Cartoonist-Hand/SF_Cartoonist_Hand.ttf'
                mutable_message = word_wrap(description,
                                            draw,
                                            text,
                                            description.width - 2*BOX_X_PADDING,
                                            description.height)
                
                draw.push()
                draw.fill_color = '#E0E094'
                draw.stroke_width = 4
#                 draw.stroke_color = 'black'
                draw.pop()
                draw.fill_color = '#004953'
                metrics = draw.get_font_metrics(description, mutable_message, multiline=True)
                draw.text(BOX_X_PADDING, int((description.height - metrics.text_height) /2 + 10), mutable_message)
#                 draw.text(BOX_X_PADDING, 10, mutable_message)
                draw(description)
#                 fg_img.border("black", 4, 4) 
#                 description.border("#367588", 4, 4) 

                with Image(width=fg_img.width+4, height=fg_img.height + description.height + 10 + 2) as output:
                    with Image(width=fg_img.width, height=fg_img.height, background='black') as tmp:
                        output.composite(image=tmp, left=4, top=2)
                    output.composite(image=fg_img, left=0, top=0)
                    
                    with Image(width=description.width, height=description.height, background='black') as tmp:
                        output.composite(image=tmp, left=4, top=fg_img.height + 10 + 2)
                    output.composite(image=description, left=0, top=fg_img.height + 10)
                    fg_img = output.clone()            

#         fg_img.border("white", 4, 4)
        image.composite(fg_img, gravity='center')
        image.border("black", 2, 2)
        image.border("#D9B382", 10, 10)
#         image.resize(int(image.width*0.5), int(image.height*0.5))
        out = save_path+str(i)+".png"
        image.save(filename=out)
        return out

In [ ]:
import os, glob, random
out_path = "C:/Users/Mcian/OneDrive/Desktop/Finalized_NFTs/"
[os.remove(p) for p in glob.glob(os.path.join(out_path, '*.png'))]
[os.remove(p) for p in glob.glob(os.path.join("C:/Users/Mcian/OneDrive/Desktop/Finalized_NFTs_metadata/", '*.png'))]
files = glob.glob(os.path.join("C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS/", '*.png'))
first = files[0]
files = files[1:]
random.shuffle(files)
files = [first] + files

In [615]:
traits = json.load(open("traits.json"))

In [619]:
names, personalities = list(traits.keys()), list(traits.values())

In [620]:
for idx, filename in enumerate(files):
    get_card(filename, out_path, idx, names[idx], personalities[idx])
    print(filename)

C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\0.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Blue bird_seed4081_guidance10.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Rabbit_seed4069_guidance4.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Red panda_seed678_guidance12.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Walrus_seed9301_guidance10.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Arctic Wolf_seed3025_guidance15.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Giant panda_seed7189_guidance7.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Beaver_seed618_guidance15.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Mastodon_seed8196_guidance2.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Galliform_seed562_guidance15.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Badger_seed9255_guidance18.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Blue whale_seed616_guidance9.png
C:/Users/Mcian/OneDrive/Desktop/Trader_NFTS\Canid_seed9011_guidance5.png
C:/Users/Mcian/OneDrive/Desktop/Tra

In [621]:
import json
for idx, filename in enumerate(glob.glob(os.path.join("C:/Users/Mcian/OneDrive/Desktop/Finalized_NFTs/", '*.png'))):
    metadata = {}
    metadata['name'] = names[idx]
    metadata['description'] = personalities[idx]
    metadata['image'] = f"ipfs://bafybeid6nfss3k5dwcfvo2x5dzfug6d3lszgm2vaasaoobtd2wulpxrjkm/{idx}.png"
    json.dump(metadata, open(f"C:/Users/Mcian/OneDrive/Desktop/Finalized_NFTs_metadata/{idx}.json", "w"))

In [661]:
out_files = glob.glob(os.path.join(out_path, '*.png'))
img = Image(filename=out_files[0])
percentage = 0.5
img.resize(int(img.width*percentage), int(img.height*percentage))
with Image(width=img.width*13, height=img.height*8) as collage:
    for i, f in enumerate(out_files):
        print(i)
        img = Image(filename=f)
        img.resize(int(img.width*percentage), int(img.height*percentage))
        collage.composite(image=img, left=img.width*(i%13), top=img.height*(i//13))
#         display(collage)
        collage.save(filename="collage.png")